In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import pandas as pd
from scipy import interpolate, stats

## Deriving the Equation of the Line

In general, we have some line of best fit $y$ given by:

$$y = a + bx$$

If we have some set of points $(x_1, y_1), (x_2, y_2), (x_3, y_3)...(x_n, y_n)$.  We need to minimize the sum of squares of residuals here, so we would have a number of values determined by:

$$[y_1 - (a + bx_1)]^2 + [y_2 - (a + bx_2)]^2 + [y_3 - (a + bx_3)]^2 + ... $$

which we can rewrite in summation notation as 

$$\sum_{i=1}^n[y_i - (a + bx_i)]^2$$

We can consider this as a function in terms of the variable $a$ that we are seeking to minimize.

$$g(a) = \sum_{i=1}^n[y_i - (a + bx_i)]^2$$

From here, we can apply our familiar strategy of differentiating the function and locating the critical values.  We are looking for the derivative of a sum, which turns out to be equivalent to the sum of the derivatives, hence we have

$$g'(a) = \sum_{i=1}^n \frac{d}{da}[y_i - (a + bx_i)]^2$$

$$g'(a) = \sum_{i=1}^n 2[y_i -a - bx_i](-1)$$

$$g'(a) = -2 [\sum_{i = 1}^n y_i - a - b\sum_{i=1}^n x_i]$$

Setting this equal to zero and solving for $a$ we get

$$a = \frac{1}{n} \sum_{i=1}^n y_i - b\frac{1}{n} \sum_{i=1}^n x_i$$

The terms should be familiar as averages, and we can rewrite our equation as

$$a = \bar{y} - b \bar{x}$$

We now use this to investigate a similar function in terms of $b$ to complete our solution.

$$f(b) = \sum_{i=1}^n[y_i - (\bar{y} + b(x_i - \bar{x}))]^2$$

We end up with 

$$b = \sum_{i = 1}^n \frac{(x_i - \bar{x})(y_i - \bar{y})}{(\bar{x} - x_i)^2}$$

Let's return to the problem of cigarette consumption and test our work out by manually computing $a$ and $b$.

## Other Situations

Our goal with regression is to identify situations where regression makes sense, fit models and discuss the reasonableness of the model for describing the data.  Data does not always come in linear forms however.  

We can easily generate sample data for familiar curves.  First, we can make some lists of polynomial form, then we will add some noise to these, fit models with `np.polyfit()`, and plot the results.  




### Non-Linear Functions

Plotting and fitting non-linear functions follows a similar pattern, however we need to take into consideration the nature of the function.  First, if we see something following a polynomial pattern, we can just use whatever degree polynomial fit we believe is relevant.  The derivation of these formulas follows the same structure as the linear case, except you are replacing the line $a - bx_i$ with a polynomial $a + bx_i + cx_i^2...$.

If we believe there to be an exponential fit, we can transform this into a linear situation using the logarithm.  For example, suppose we have the following population data.

| Decade $t$ | Year | Population |
| ----- | ------ | ----- |
| 0 | 1780 | 2.8 | 
| 1 | 1790 | 3.9 | 
| 2 | 1800 | 5.3 | 
| 3 | 1810 | 7.2 |

If we examine the data, we see an exponential like trend.  If we use NumPy to find the logarithm of the population values and plot the result, we note the transformed datas similarity to a linear function.

In [ ]:
t = np.arange(0,13)
year = np.arange(1780,1910,10)
P = [2.8, 3.9, 5.3, 7.2, 9.6, 12.9, 17.1, 23.2, 31.4, 39.8, 50.2, 62.9, 76.0]

In [ ]:
plt.figure(figsize = (7,5))
plt.subplot(1, 2, 1)
plt.scatter(year, P,color = 'red', alpha = 0.7)
plt.xlabel('Year')
plt.ylabel('Population')
plt.title("Original Data")

plt.subplot(1, 2, 2)
lnP = np.log(P)
plt.scatter(year, lnP, color = 'green', alpha = 0.4)
plt.xlabel('Year')
plt.ylabel('Logarithm of Population')
plt.title("Transformed Data")

Symbolically, we would imagine the original function as an exponential of the form

$$y = ae^{bx}$$

The expression can be explored in a similar manner, where we use Sympy to find the effect of the logarithm.

In [ ]:
y, a, b, x = sy.symbols('y a b x')

In [ ]:
eq = sy.Eq(y, a*sy.exp(b*x))

In [ ]:
sy.expand_log(sy.log(b**x))

In [ ]:
sy.expand_log(sy.log(a*sy.exp(b*x)), force = True)

Hence, we have that

$$\log(y) = bx + \log(a)$$

which should look like our familiar linear equations.  Here, we can find $a$ and $b$, then convert the equation back to its original form by undoing the logarithm with the exponential.

For kicks, we introduce the SciPy `linregress` function.  Feel free to examine the help documentation for the function.  This gives a little more information about the model than the `polyfit` function.  Further, we add text to the plot to display information about the model.

In [ ]:
line = np.polyfit(year, lnP, 1)
fit = np.polyval(line, year)
alpha, beta, r_value, p_value, std_err = stats.linregress(year, lnP) #
alpha, beta, r_value

In [ ]:
fig = plt.figure(figsize = (10,5))

fig.add_subplot(121)
plt.plot(year, np.exp(fit))
plt.plot(year, P, 'o', markersize = 7, alpha = 0.8)

fig.add_subplot(122)
plt.plot(year, fit)
plt.plot(year, lnP, 'o', markersize = 7, alpha = 0.8)

## Logistic Example

As you see above, towards the end of our model the actual and predicted values seem to diverge.  Considering the context, this makes sense.  A population should reach some maximum levels due to physical resources.  A more S shaped curve is the logistic function which is given by 

$$y = \frac{L}{1 + e^{a+bx}}$$

As an example, consider the Inter Continental Ballistic Missle Data for 1960 - 1969.

| Year | Number of ICBM's |
| --- | --- |
| 1960 | 18 |
| 1961 | 63 |
| 1962 | 294 |
| 1963 | 424 |
| 1964 | 834 |
| 1965 | 854 |
| 1966 | 904 |
| 1967 | 1054 |
| 1968 | 1054 |
| 1969 | 1054 |

In [ ]:
year = [i for i in np.arange(1960, 1970, 1)]
icbm = [18, 63, 294, 424, 834, 854, 904, 1054, 1054, 1054]

In [ ]:
plt.scatter(year, icbm)

In [ ]:
L, y, a, b, x = sy.symbols('L y a b x')

In [ ]:
exp = sy.Eq(y, L/(1 + sy.exp(a + b*x)))

In [ ]:
sy.solve(exp, (a + b*x),  force = True)

This means that the tranformation that linearizes our data is 

$$\log(\frac{L - y}{y})$$

The value $L$ is defined as the *carrying capacity* of the model.  Here, it seems something like $L = 1060$ would be a reasonable value to try.  

In [ ]:
t_icbm = [np.log((1060 - i)/i) for i in icbm]

In [ ]:
plt.scatter(year, t_icbm)

In [ ]:
b, a = np.polyfit(year, t_icbm, 1)

In [ ]:
a, b

In [ ]:
def l(x):
    return b*x + a

l(1960), l(1969)

In [ ]:
fit = [l(i) for i in year]

In [ ]:
plt.scatter(year, t_icbm)
plt.plot(year, fit)
plt.title("Fitting ICBM Data")
plt.xlabel("Year")
plt.ylabel("Transformed ICMB Data")

Much like the last example, we can return everything to its original form with the exponential.  We arrive at the equation

$$y = \frac{1060}{1 + e^{2092 - 1.0654x}}$$

In [ ]:
def y(x):
    return 1060/(1 + np.exp(2092 - 1.0654*x))

o_fit = [y(i) for i in year]

In [ ]:
plt.scatter(year, icbm)
plt.plot(year, o_fit, '--k')
plt.title("ICBM Data and Logistic Fit")
plt.xlabel("Year")
plt.ylabel("ICMB's")

### Example

The dataset below contains historical data dealing with some basic human development indicies.  Your goal is to explore the relationship between life expectancy and GDP.  Examine a linear regression model that you build on the most recent data comparing GDP per capita and life expectancy.

In [ ]:
gap = pd.read_csv('data/gapminder_all.csv')

In [ ]:
gap.head()